In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers
from tensorflow.keras import Input
from absl import logging
import matplotlib.pyplot as plt
import re
import glob
import json
import tensorflow_hub as hub

In [3]:
original_data = pd.read_csv("data/mbti_1.csv")
def remove_links(posts):
    return [re.sub("(?P<url>https?://[^\s]+)", '', post) for post in posts.split('|||')]
original_data['posts'] = original_data['posts'].apply(remove_links)
types = pd.unique(original_data['type'])
code = {tp:i for i,tp in enumerate(types)}
rev_code  = {i:tp for i,tp in enumerate(types)}

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embedding_model = hub.load(module_url)
def embed(input):
  return embedding_model(input)

In [5]:
def embed_user_posts(posts):
    return np.array(embed(posts))
def convert_type(ty):
    return code[ty]
original_data['posts'] = original_data['posts'].apply(embed_user_posts)
original_data['type'] = original_data['type'].apply(convert_type)
original_data.head()

,type,posts
0,0,"[[-0.035656825, -0.033195727, 0.0704743, 0.073..."
1,1,"[[0.006778221, 0.045070868, -0.011752028, -0.0..."
2,2,"[[-0.009484898, -0.003131814, 0.0037449605, -0..."
3,3,"[[-0.009333394, -0.068066284, 0.024093557, 0.0..."
4,4,"[[-0.038833104, -0.038577285, 0.025958145, 0.0..."


In [6]:
for i, person in enumerate(original_data['posts']):
    if person.shape != (50,512):
        original_data = original_data.drop([i])
original_data.shape

(7587, 2)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(np.stack(original_data['posts'].values),np.expand_dims(np.array(original_data['type']),axis=1),test_size=.1)

In [15]:
model = Sequential()
model.add(layers.Conv1D(64,3,input_shape=(50,512)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(.4))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(16,activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=15,steps_per_epoch=1028,validation_data=(X_test,y_test))

Epoch 1/15
1028/1028 [==============================] - 3s 3ms/step - loss: 2.2268 - accuracy: 0.2509 - val_loss: 1.9890 - val_accuracy: 0.2991
Epoch 2/15
1028/1028 [==============================] - 3s 2ms/step - loss: 1.8271 - accuracy: 0.3846 - val_loss: 1.9054 - val_accuracy: 0.3531
Epoch 3/15
1028/1028 [==============================] - 3s 2ms/step - loss: 1.5734 - accuracy: 0.4666 - val_loss: 2.3797 - val_accuracy: 0.3794
Epoch 4/15
1028/1028 [==============================] - 3s 2ms/step - loss: 1.2208 - accuracy: 0.5880 - val_loss: 2.8773 - val_accuracy: 0.3083
Epoch 5/15
1028/1028 [==============================] - 3s 2ms/step - loss: 0.8491 - accuracy: 0.7083 - val_loss: 3.5077 - val_accuracy: 0.3347
Epoch 6/15
1028/1028 [==============================] - 3s 2ms/step - loss: 0.6351 - accuracy: 0.7925 - val_loss: 4.5932 - val_accuracy: 0.3004
Epoch 7/15
 792/1028 [======================>.......] - ETA: 0s - loss: 0.5867 - accuracy: 0.8208

KeyboardInterrupt: 

In [ ]:
results = [rev_code[i] for i in np.argmax(model.predict(X_test),axis=1)]
pd.Series(results).value_counts().plot(kind='bar')

In [ ]:
pd.Series([rev_code[int(i)] for i in y_test]).value_counts().plot(kind='bar')